## Getting Market Component GeoLocation

#### Importing Dependencies

In [1]:
import tradingeconomics as te
import pandas as pd
import numpy as np
import pycountry
import plotly.graph_objects as go
import plotly.express as px

from urllib.parse import quote
from tradingeconomics import functions as fn
from tradingeconomics import glob as glob

#### API login


In [48]:
te.login('guest:guest') # Signup for a free API key at https://developer.tradingeconomics.com/

'You are logged in as a guest:guest'

### Importing Data

#### Choose an argument to parameter 'symbol'
    examples:
    - 'FTSEMIB:IND'
    - 'UKX:IND'
    - 'SPX:IND'
    - 'INDEXCF:IND'
    

In [41]:
symbol = ['SPX:IND']
df = te.getMarketsComponents(symbols=symbol, output_type='df')

df[['Symbol','Name', 'Country']]


,Symbol,Name,Country
0,A:US,Agilent,United States
1,AA:US,Alcoa,United States
2,AAL:US,American Airlines,United States
3,AAP:US,Advance Auto Parts,United States
4,AAPL:US,Apple,United States
...,...,...,...
524,YUM:US,Yum Brands,United States
525,ZBH:US,Zimmer Biomet Holdings Inc,United States
526,ZBRA:US,Zebra Technologies,United States
527,ZION:US,Zions Bancorporation,United States


#### 'Group By' to get companies number by country

In [42]:
df4 = df.groupby('Country')['Country'].agg(['count'])
df4

,count
Country,
Bermuda,1
China,1
France,1
Ireland,9
Netherlands,2
Singapore,1
Switzerland,3
United Kingdom,7
United States,504


#### Creates a dataframe to receive Country and number of companies

In [43]:
df5 = pd.DataFrame()
df5['Country'] =df4['count'].keys()
df5['Companies'] = df4['count'].values
df5

,Country,Companies
0,Bermuda,1
1,China,1
2,France,1
3,Ireland,9
4,Netherlands,2
5,Singapore,1
6,Switzerland,3
7,United Kingdom,7
8,United States,504


### Parsing Country Name to Country alpha_iso3 symbol



In [44]:
countriesWithoutISO = []

missing_countries = {
    'Bolivia' : 'Bolivia, Plurinational State of',
'Cape Verde' : 'Cabo Verde',
'Cayman Islands      '  : 'Cayman Islands',   
'Czech Republic' : 'Czechia',
'European Union' : 'European Union',
"Ivory Coast" : "Côte d'Ivoire",
'Laos' : "Lao People's Democratic Republic",
'Macau' : 'Macao',
'Macedonia' : 'North Macedonia',
'Moldova' : 'Moldova, Republic of',
'Norway              '   : 'Norway',            
'Republic of the Congo' : 'Congo, The Democratic Republic of the',
'Russia' : 'Russian Federation',
'Singapore           '  : 'Singapore',          
'South Korea' : 'Korea, Republic of',
'St Vincent and the Grenadines' : 'Saint Vincent and the Grenadines',
'Swaziland' : 'Eswatini',
'Sweden              ' : 'Sweden',              
'Taiwan' : 'Taiwan, Province of China',
'Tanzania' : 'Tanzania, United Republic of',
'Turkey ': 'Turkey', 
'Venezuela' : 'Venezuela, Bolivarian Republic of',
'Vietnam' : 'Viet Nam',
'United States ': 'United States',
'Brunei': 'Brunei Darussalam',
    'East Timor': 'Timor-Leste',
    'Guinea Bissau': 'Guinea-Bissau',
    'Iran': 'Iran, Islamic Republic of',
    'North Korea': "Korea, Democratic People's Republic of",
    'Palestine': "Palestine, State of",
    'St Kitts and Nevis': 'Saint Kitts and Nevis',
    'St Lucia': 'Saint Lucia',
    'St Vincent And The Grenadines': 'Saint Vincent and the Grenadines'
}

def get_iso3(country_full_name):
            """
                returns country symbol iso alpha 3 format 
            """
            symbol = None
            iso = pycountry.countries.get(name=country_full_name)
            if iso != None:
                symbol = iso.alpha_3
            else:
                if country_full_name not in countriesWithoutISO:
                    countriesWithoutISO.append(country_full_name)
                    print('Failed to get ISO for: ' + country_full_name)
            return symbol



### Treating the data

In [45]:
iso3_countries_names = []
corrected_countries_names_list = df5['Country'].replace(missing_countries)   
df_corrected_countries_names = pd.DataFrame(corrected_countries_names_list)
df_corrected_countries_names['Companies'] = df5['Companies']

for item in df_corrected_countries_names['Country']:
    iso3_countries_names.append(get_iso3(item))

df_corrected_countries_names['iso3'] = iso3_countries_names
df_corrected_countries_names

,Country,Companies,iso3
0,Bermuda,1,BMU
1,China,1,CHN
2,France,1,FRA
3,Ireland,9,IRL
4,Netherlands,2,NLD
5,Singapore,1,SGP
6,Switzerland,3,CHE
7,United Kingdom,7,GBR
8,United States,504,USA


### Plotting

In [47]:

fig = px.choropleth(df_corrected_countries_names, locations="iso3",title= f"Market Component for {symbol}" ,
                    color="Companies", 
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()